Лабораторная работа 7

In [1]:
import xlwings as xw
from xlwings.constants import AutoFillType
import pandas as pd

In [2]:
path1 = "C:\Мари\учеба\Финансовый университет\\1 курс\\2 семестр\Технологии обработки данных\Лабораторная работа 4\data\\"
path2 = "C:\Мари\учеба\Финансовый университет\\1 курс\\2 семестр\Технологии обработки данных\Лабораторная работа 5\data\\"
path3 = "C:\Мари\учеба\Финансовый университет\\1 курс\\2 семестр\Технологии обработки данных\Лабораторная работа 2\data\\"

1. Загрузите данные из файлов reviews_sample.csv (ЛР2) и recipes_sample_with_tags_ingredients.csv (ЛР5) в виде pd.DataFrame. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: id, name, minutes, submitted, description, n_ingredients

In [3]:
reviews_sample = pd.read_csv(path3 + 'reviews_sample.csv', sep=',')
reviews_sample = reviews_sample.set_index('Unnamed: 0')
print(reviews_sample)

               user_id  recipe_id        date  rating  \
Unnamed: 0                                              
370476           21752      57993  2003-05-01       5   
624300          431813     142201  2007-09-16       5   
187037          400708     252013  2008-01-10       4   
706134      2001852463     404716  2017-12-11       5   
312179           95810     129396  2008-03-14       5   
...                ...        ...         ...     ...   
1013457        1270706     335534  2009-05-17       4   
158736         2282344       8701  2012-06-03       0   
1059834         689540     222001  2008-04-08       5   
453285      2000242659     354979  2015-06-02       5   
691207          463435     415599  2010-09-30       5   

                                                       review  
Unnamed: 0                                                     
370476      Last week whole sides of frozen salmon fillet ...  
624300      So simple and so tasty!  I used a yellow capsi...  
18

In [4]:
recipes_sample_with_tags_ingredients = pd.read_csv(path2 + 'recipes_sample.csv', sep=',')
recipes_sample_with_tags_ingredients = recipes_sample_with_tags_ingredients[
    ['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']
]
print(recipes_sample_with_tags_ingredients)

           id                                          name  minutes  \
0       44123         george s at the cove  black bean soup       90   
1       67664            healthy for them  yogurt popsicles       10   
2       38798                  i can t believe it s spinach       30   
3       35173                          italian  gut busters       45   
4       84797      love is in the air  beef fondue   sauces       25   
...       ...                                           ...      ...   
29995  267661  zurie s holey rustic olive and cheddar bread       80   
29996  386977          zwetschgenkuchen  bavarian plum cake      240   
29997  103312   zwiebelkuchen   southwest german onion cake       75   
29998  486161                                   zydeco soup       60   
29999  298512        cookies by design   cookies on a stick       29   

        submitted                                        description  \
0      2002-10-25  an original recipe created by chef scott mes

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл recipes.xlsx. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц.

In [5]:
recipes = pd.read_csv(path2 + 'recipes_sample.csv')
reviews = pd.read_csv(path3 + 'reviews_sample.csv')

In [6]:
data1 = reviews.sample(frac=0.05, random_state=42)
data2 = recipes.sample(frac=0.05, random_state=42)

In [7]:
wb1 = xw.Book()
wb1.save(path1 + 'recipes.xlsx')

In [8]:
sheet1 = wb1.sheets.add('Отзывы')
sheet2 = wb1.sheets.add('Рецепты')

In [9]:
sheet1.range('A1').value = data1
sheet2.range('A1').value = data2
sheet1.range('A:A').api.Delete()
sheet2.range('A:A').api.Delete()

True

3. Используя xlwings, добавьте на лист Рецепты столбец seconds_assign, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [10]:
seconds_assign = (data2["minutes"] * 60).to_numpy()
print(len(seconds_assign))

1500


In [11]:
sheet2.range('G1').value = 'seconds_assign'
sheet2.range('G2').options(transpose=True).value = seconds_assign

4. Используя xlwings, добавьте на лист Рецепты столбец seconds_formula, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [12]:
sheet2.range('H1').value = 'seconds_formula'
sheet2.range('H2').formula = '=B2*60'
sheet2.range('H2').api.AutoFill(sheet2.range('H2:H{}'.format(len(seconds_assign) + 1)).api, 0)

True

5. Добавьте на лист Рецепты столбец n_reviews, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [13]:
sheet2.range('I1').value = 'n_reviews'
sheet2.range('I2').formula = '=COUNTIF(Отзывы!$C:$C, C2)'
sheet2.range('I2').api.AutoFill(sheet2.range('I2:I{}'.format(len(data2) + 1)).api, 0)

True

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [14]:
sheet1.range('G1:E1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
sheet1.range('G1:E1').api.Font.Bold = True

In [15]:
sheet2.range('G1:I1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
sheet2.range('G1:I1').api.Font.Bold = True

7. Раскрасьте ячейки столбца minutes в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [16]:
for cell in sheet2.range(f'C2:C{len(data2) + 1}'):
    if cell.value < 5:
        cell.color = (0, 255, 0)
    elif cell.value >= 5 and cell.value <= 10:
        cell.color = (250, 255, 0)
    else:
        cell.color = (255, 0, 0)

com_error: (-2146827864, 'OLE error 0x800a01a8', None, None)

8. Напишите функцию validate(), которая проверяет соответствие всех строк из листа Отзывы следующим правилам:

Рейтинг - это число от 0 до 5 включительно

Соответствующий рецепт имеется на листе Рецепты

В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate():
    wb = xw.Book.caller()
    reviews_sheet = wb.sheets['Отзывы']
    recipes_sheet = wb.sheets['Рецепты']

    for row in range(2, reviews_sheet.range('A1').current_region.last_cell.row + 1):
        rating = reviews_sheet.range('E{}'.format(row)).value
        recipe_id = reviews_sheet.range('C{}'.format(row)).value
        recipe_match = recipes_sheet.range('B:C').value == recipe_id
        if not isinstance(rating, (int, float)) or rating < 0 or rating > 5 or not any(recipe_match):
            reviews_sheet.range('A{}:D{}'.format(row, row)).color = (255, 0, 0)